In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, UpSampling2D,concatenate, Dense
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2DTranspose, Cropping2D

In [7]:
def unet(input_shape):
    # Input layer
    inputs = Input(shape=input_shape)

    # Downsample path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Upsample path
    up6 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv2)
    conv6 = Conv2D(64, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(1, 1, activation='sigmoid')(conv6)  # Output a single-channel binary mask

    # Continue with the remaining layers of the U-Net architecture

    # Create the model
    model = Model(inputs=inputs, outputs=conv6)
    return model

# Define the input shape
input_shape = (200, 200, 3)

# Create the U-Net model
model = unet(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 200, 200, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 200, 200, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 100, 100, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 100, 100, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 100, 128)     147584    
                                                             

In [13]:
# Define the directory path containing the images
directory_path = '/Users/ouzibigouziouzi/Desktop/FNNPK'

# Iterate through all the files and subdirectories in the directory
for root, dirs, files in os.walk(directory_path):
    for file in files:
        # Check if the file has an image extension (e.g., .jpg, .png)
        if file.endswith(('.jpg', '.png')):
            # Construct the file path
            file_path = os.path.join(root, file)

            # Load the input image
            image = cv2.imread(file_path)

            # Resize the image to match the input shape of the model
            image = cv2.resize(image, (200, 200))

            # Perform image segmentation
            segmentation_mask = model.predict(np.expand_dims(image, axis=0))

            # Convert the segmentation mask to a binary mask
            binary_mask = np.where(segmentation_mask > 0.5, 1, 0)

            # Resize the binary mask to match the shape of the original image using interpolation
            binary_mask = cv2.resize(binary_mask[0], (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

            # Expand dimensions of the binary mask to match the number of channels in the image
            binary_mask = np.expand_dims(binary_mask, axis=2)

            # Apply the binary mask to the original image
            foreground = image * binary_mask

            # Create a black background
            background = np.zeros_like(image)

            # Generate the final background-removed image
            background_removed_image = np.where(binary_mask == 1, foreground, background)

            # Construct the output file path in the output directory with the same name as the input file
            output_file_path = os.path.join('/Users/ouzibigouziouzi/Desktop/nf_Hydroponics', file)

            # Save the background-removed image in the output directory with the same name
            cv2.imwrite(output_file_path, background_removed_image)

1/1 [==============================] - 0s 65ms/step
